In [41]:
from gensim.models import fasttext as ft
import numpy as np
import os
import nltk
from nltk.corpus import stopwords

In [45]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yoshi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
# https://fasttext.cc/docs/en/crawl-vectors.html
model=ft.load_facebook_model(os.path.join(os.getcwd(),"binaries/data/cc.en.300.bin"))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-46-51c1064b2318>", line 2, in <module>
    model=ft.load_facebook_model(os.path.join(os.getcwd(),"binaries/data/cc.en.300.bin"))
  File "C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\models\fasttext.py", line 1250, in load_facebook_model
    return _load_fasttext_format(path, encoding=encoding, full_model=True)
  File "C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\models\fasttext.py", line 1330, in _load_fasttext_format
    m = gensim.models._fasttext_bin.load(fin, encoding=encoding, full_model=full_model)
  File "C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\

KeyboardInterrupt: 

In [0]:
window_size=3
stemmer = nltk.wordnet.WordNetLemmatizer()

In [0]:
def rank(text):
    stops = set(stopwords.words("english"))
    tokens = nltk.word_tokenize(text)
    filtered_words = [stemmer.lemmatize(word) for word in tokens if word not in stops]
    scores=dict()
    for i in range(len(filtered_words)):
        for j in range(min(len(filtered_words),i+3)):
            scores.setdefault(filtered_words[i],0)
            scores.setdefault(filtered_words[j],0)
            similarity_score=model.similarity(filtered_words[i],filtered_words[j])
            scores[filtered_words[i]]+=similarity_score
            scores[filtered_words[j]]+=similarity_score
    return sorted(scores)

In [0]:
rank("Word embedding is the collective name for a set of language modeling and feature learning techniques in natural language processing (NLP) where words or phrases from the vocabulary are mapped to vectors of real numbers. Conceptually it involves a mathematical embedding from a space with many dimensions per word to a continuous vector space with a much lower dimension.")